In [ ]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.FIM import FIMModel

In [ ]:
hparams = {
    'scale':'demo',
    'name':'fim',
    'mode':'train',
    'epochs':1,
    'batch_size':100,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'k':-1,
    'save_step':[0],
}

In [ ]:
vocab, loaders = prepare(hparams)

In [ ]:
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(hparams['device'])

In [ ]:
train(fimModel, hparams, loaders, interval=10)

In [ ]:
evaluate(fimModel, hparams, loaders[1])